In [1]:
import os, joblib
import pandas as pd
import numpy as np

from utils.Dataset import CraneDataset
from sklearn.ensemble import RandomForestRegressor

In [2]:
data_file_path_list = []
data_file_name_list = os.listdir('data')

for file_name in data_file_name_list:
    data_file_path_list.append(os.path.join('data', file_name))

In [3]:
dataset_inst = CraneDataset(data_file_path_list)
raw_dataset = dataset_inst.get_dataset()

feature_names = dataset_inst.get_data_feature_names()
target_names = dataset_inst.get_data_target_names()

for t_name in target_names:
    raw_dataset[t_name] = raw_dataset[t_name]/1000

for t_name in target_names:
    raw_dataset = raw_dataset[raw_dataset[t_name]>=1]
    
#raw_dataset = raw_dataset[raw_dataset['Time(sec)']>0]

raw_dataset.reset_index(drop=True, inplace=True)

extra_feature_name_list = ['boom+roll/load', 'boom+pitch/load', 'swing/load', 'roll/pitch', 'load/roll', 'load/pitch', 'boom+roll', 'boom+pitch', 'boom_x_pos', 'boom_y_pos']

raw_dataset['boom+roll/load'] = (raw_dataset['Boom_Angle(deg)']+raw_dataset['Roll_Angle(deg)'])/raw_dataset['Load(Ton)']
raw_dataset['boom+pitch/load'] = (raw_dataset['Boom_Angle(deg)']+raw_dataset['Pitch_Angle(deg)'])/raw_dataset['Load(Ton)']
raw_dataset['swing/load'] = raw_dataset['Swing_Angle(deg)']/raw_dataset['Load(Ton)']
raw_dataset['roll/pitch'] = raw_dataset['Roll_Angle(deg)']/raw_dataset['Pitch_Angle(deg)']
raw_dataset['load/roll'] = raw_dataset['Load(Ton)']/raw_dataset['Roll_Angle(deg)']
raw_dataset['load/pitch'] = raw_dataset['Load(Ton)']/raw_dataset['Pitch_Angle(deg)']
raw_dataset['boom+roll'] = raw_dataset['Boom_Angle(deg)'] + raw_dataset['Roll_Angle(deg)']
raw_dataset['boom+pitch'] = raw_dataset['Boom_Angle(deg)'] + raw_dataset['Pitch_Angle(deg)']
raw_dataset['boom_x_pos'] = np.cos(np.deg2rad(raw_dataset['Swing_Angle(deg)'])) * np.cos(np.deg2rad(raw_dataset['Boom_Angle(deg)']))
raw_dataset['boom_y_pos'] = np.sin(np.deg2rad(raw_dataset['Swing_Angle(deg)'])) * np.cos(np.deg2rad(raw_dataset['Boom_Angle(deg)']))
    
diff_boom_angle = np.diff(raw_dataset['Boom_Angle(deg)'])
diff_swing_angle = np.diff(raw_dataset['Swing_Angle(deg)'])
diff_roll_angle = np.diff(raw_dataset['Roll_Angle(deg)'])
diff_pitch_angle = np.diff(raw_dataset['Pitch_Angle(deg)'])

#extra_feature_name_list = ['grad_boom_angle', 'grad_swing_angle', 'grad_roll_angle', 'grad_pitch_angle']
#new_feature_names = feature_names + extra_feature_name_list

#for extra_feature_name, extra_feature  in zip(extra_feature_name_list, [grad_boom_angle, grad_swing_angle, grad_roll_angle, grad_pitch_angle]):
#    raw_dataset[extra_feature_name] = extra_feature

train_dataset_1 = raw_dataset[raw_dataset['dataset_type'] == 0]
train_dataset = train_dataset_1.drop(columns=['Time(sec)', 'file_idx', 'safe_state', 'dataset_type'])
train_dataset = train_dataset[feature_names[0:4]+[feature_names[5]]+extra_feature_name_list+target_names]
train_dataset.reset_index(drop=True, inplace=True)

val_dataset_1 = raw_dataset[raw_dataset['dataset_type'] == 1]
val_dataset = val_dataset_1.drop(columns=['Time(sec)', 'file_idx', 'safe_state', 'dataset_type'])
val_dataset = val_dataset[feature_names[0:4]+[feature_names[5]]+extra_feature_name_list+target_names]
val_dataset.reset_index(drop=True, inplace=True)

  0%|          | 0/22 [00:00<?, ?it/s]

In [4]:
train_dataset.describe()

,Boom_Angle(deg),Swing_Angle(deg),Load(Ton),Roll_Angle(deg),Pitch_Angle(deg),boom+roll/load,boom+pitch/load,swing/load,roll/pitch,load/roll,...,Actual_Load_Left_1(N),Actual_Load_Left_2(N),Actual_Load_Left_3(N),Actual_Load_Left_4(N),Actual_Load_Left_5(N),Actual_Load_Right_1(N),Actual_Load_Right_2(N),Actual_Load_Right_3(N),Actual_Load_Right_4(N),Actual_Load_Right_5(N)
count,76951.000000,7.695100e+04,76951.000000,76951.000000,76951.000000,76951.000000,76951.000000,7.695100e+04,76951.000000,7.695100e+04,...,76951.000000,76951.000000,76951.000000,76951.000000,76951.000000,76951.000000,76951.000000,76951.000000,76951.000000,76951.000000
mean,65.244964,7.148414e+01,75.703889,-0.000095,-0.000082,0.926743,0.926743,1.049855e+00,11.037170,-6.743289e+06,...,294.399906,279.314035,260.959760,242.051725,225.310701,228.699350,209.834585,190.629113,173.742344,161.857265
std,10.838993,6.478831e+01,25.525701,0.000188,0.000289,0.232774,0.232773,1.083413e+00,3425.251487,2.183465e+09,...,118.629640,78.395474,54.837048,79.415485,121.068184,120.196203,75.108241,42.903806,67.080940,109.747233
min,40.000000,-3.377295e-23,40.000000,-0.014762,-0.009508,0.569282,0.569291,-3.070268e-25,-15267.807788,-5.975398e+11,...,1.009678,40.448960,22.720991,8.589950,1.018398,1.036816,2.564975,4.721832,7.326577,1.031677
25%,60.000000,5.089531e+00,50.000000,-0.000183,-0.000290,0.699999,0.700000,6.704352e-02,-0.194376,-1.112839e+06,...,204.975095,214.508315,211.565825,176.642020,121.081860,121.555835,146.409220,159.889050,121.717605,66.698930
50%,70.000000,5.331334e+01,70.000000,-0.000052,-0.000055,0.987527,0.987524,7.800375e-01,-0.020052,-3.819575e+05,...,317.556440,275.174630,251.874020,234.321110,196.912580,221.077950,207.986540,196.576770,166.371030,136.465460
75%,70.000000,1.350000e+02,100.000000,0.000002,0.000127,1.000000,1.000004,1.669641e+00,0.355243,2.230936e+06,...,393.593530,342.354515,295.611795,305.811600,333.727170,324.852970,265.491480,219.180720,206.399405,237.940900
max,80.000000,1.800000e+02,120.000000,0.009374,0.015054,1.749991,1.749992,4.500000e+00,949563.561484,4.371400e+10,...,500.149710,500.129210,500.156300,500.203910,500.251520,500.161150,500.128300,500.131300,500.134300,500.137300


In [5]:
est_model = RandomForestRegressor(n_estimators=50, max_features='log2', random_state=0, n_jobs=os.cpu_count())

target_start_idx = 15
est_model.fit(train_dataset.iloc[:, 0:target_start_idx].to_numpy(), train_dataset.iloc[:, target_start_idx:].to_numpy())

RandomForestRegressor(max_features='log2', n_estimators=50, n_jobs=48,
                      random_state=0)

In [7]:
pred_out = est_model.predict(val_dataset.iloc[:, 0:target_start_idx].to_numpy())

pred_target_names = ['Pred_Load_Left_1(kN)', 'Pred_Load_Left_2(kN)', 'Pred_Load_Left_3(kN)', 'Pred_Load_Left_4(kN)', 'Pred_Load_Left_5(kN)',
                     'Pred_Load_Right_1(kN)', 'Pred_Load_Right_2(kN)', 'Pred_Load_Right_3(kN)', 'Pred_Load_Right_4(kN)', 'Pred_Load_Right_5(kN)']

target_pred = pd.DataFrame(pred_out, columns=pred_target_names)

In [8]:
absolute_error_names = ['Absolute_Error_Left_1(kN)', 'Absolute_Error_Left_2(kN)', 'Absolute_Error_Left_3(kN)', 'Absolute_Error_Left_4(kN)', 'Absolute_Error_Left_5(kN)',
                        'Absolute_Error_Right_1(kN)', 'Absolute_Error_Right_2(kN)', 'Absolute_Error_Right_3(kN)', 'Absolute_Error_Right_4(kN)', 'Absolute_Error_Right_5(kN)']

relative_error_names = ['Relative_Error_Left_1(%)', 'Relative_Error_Left_2(%)', 'Relative_Error_Left_3(%)', 'Relative_Error_Left_4(%)', 'Relative_Error_Left_5(%)',
                        'Relative_Error_Right_1(%)', 'Relative_Error_Right_2(%)', 'Relative_Error_Right_3(%)', 'Relative_Error_Right_4(%)', 'Relative_Error_Right_5(%)']

actual_load = val_dataset[target_names].to_numpy()

pred_load = target_pred.to_numpy()

absolute_error = pd.DataFrame(abs(actual_load-pred_load), columns=absolute_error_names)
relative_error = pd.DataFrame((abs(actual_load-pred_load)/abs(actual_load))*100, columns=relative_error_names)
result = pd.concat([val_dataset, target_pred, absolute_error, relative_error], axis=1)

result.describe()

,Boom_Angle(deg),Swing_Angle(deg),Load(Ton),Roll_Angle(deg),Pitch_Angle(deg),boom+roll/load,boom+pitch/load,swing/load,roll/pitch,load/roll,...,Relative_Error_Left_1(%),Relative_Error_Left_2(%),Relative_Error_Left_3(%),Relative_Error_Left_4(%),Relative_Error_Left_5(%),Relative_Error_Right_1(%),Relative_Error_Right_2(%),Relative_Error_Right_3(%),Relative_Error_Right_4(%),Relative_Error_Right_5(%)
count,16300.000000,1.630000e+04,16300.000000,16300.000000,16300.000000,16300.000000,16300.000000,1.630000e+04,16300.000000,1.630000e+04,...,16300.000000,16300.000000,16300.000000,16300.000000,16300.000000,16300.000000,16300.000000,16300.000000,16300.000000,16300.000000
mean,66.990055,6.811634e+01,74.288957,-0.000122,-0.000089,0.906948,0.906948,9.139215e-01,-0.340320,5.017316e+06,...,5.205019,2.785990,2.228661,3.109366,5.326191,16.697539,10.808968,10.107830,12.436948,19.782566
std,4.254646,5.464557e+01,4.949335,0.000122,0.000238,0.096699,0.096698,7.135320e-01,107.362220,2.944373e+08,...,8.096656,2.866957,2.229069,2.616406,28.144494,58.212550,47.469740,60.996618,84.901928,138.586513
min,40.855858,-8.949622e-26,70.000000,-0.000752,-0.000971,0.583644,0.583655,-1.118703e-27,-7777.544921,-1.456265e+09,...,0.000003,0.000145,0.000614,0.000305,0.000944,0.000002,0.000739,0.000622,0.000149,0.000106
25%,65.000000,1.826267e+01,70.000000,-0.000163,-0.000225,0.812499,0.812502,2.482410e-01,-0.202995,-8.541366e+05,...,0.854792,0.864635,0.632006,1.248820,2.347668,1.642745,1.604065,1.966573,2.153236,3.449933
50%,68.825081,5.491546e+01,70.000000,-0.000100,-0.000148,0.946443,0.946444,7.335422e-01,0.057188,-5.100561e+05,...,2.156986,1.921274,1.668674,2.519788,4.423704,4.128005,4.297571,3.415089,5.356768,7.970107
75%,70.000000,1.100589e+02,80.000000,-0.000028,0.000080,0.999999,0.999998,1.436491e+00,0.673562,-2.646210e+05,...,4.760833,3.708745,2.958113,4.455341,6.411984,10.975295,8.728755,7.801180,8.638446,12.089771
max,70.000000,1.800000e+02,80.000000,0.000071,0.000420,1.000001,1.000004,2.250000e+00,2405.759432,3.126450e+10,...,62.800129,44.328735,28.900258,59.495029,2483.969325,2458.565991,1683.729494,2488.578866,4011.613525,7373.639080


In [9]:
result[relative_error_names].mean().mean()

8.848907850709965

In [10]:
joblib.dump(est_model, 'rf_model.joblib')

['rf_model.joblib']